##1. mongoDB란?

### 1.1. mongoDB 란?
* mongoDB는 document db
  - JSON 기반의 Document 기반 데이터 관리

* 프로그래밍에서 다루는 데이터 포멧
  - 정수(int), 소숫점(float), 문자(string)
  - csv, JSON등
* JSON
  document  = { "id":"01", "languange":"java", "edition": { "first": "1st", "second":"2nd", "third":"third" } }

<pre>
MongoDB Document 예)

{
    "_id": ObjectId("5099803df3f42312312391"),
    "username": "davelee",
    "name": { first: "Dave", last: "Lee" }
}
</pre>

#### MongoDB Database
* Database는 Collection의 집합

#### MongoDB Collection

* Collection은 MongoDB Document의 집합
* RDBMS Table과 유사한 개념, 단 정규화된 데이터 구조, 즉 Schema가 정의되어 있지 않음

<img src="https://www.fun-coding.org/00_Images/mongodb_mysql.png" /> 

## 2. mongoDB 설치 및 환경 구축 

### 2.1. mongodb  설치 방법 (맥 환경)

* Homebrew 프로그램을 사용해서 설치하는 것이 가장 쉬움
  - https://brew.sh/index_ko 
  - /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
  
* Homebrew 설치 후
  - brew install mongodb
  
* mongodb 설치 후
  - mongod 또는 brew services start mongodb 
  

### 2.2. mongodb  설치 방법 (윈도우 환경)

* https://www.mongodb.com/download-center/community 이동 후, .msi 파일 다운로드 및 설치
* 참고 블로그: https://javacpro.tistory.com/64

### 2.3. EC2(AWS 서버)에 mongodb  설치 방법 (ubuntu 기반)

* Import the public key(GPG key) used by the package management system.
  - sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 0C49F3730359A14518585931BC711F9BA15703C6
* Create a list file for MongoDB.
  - echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list
* Reload local package database.
  - sudo apt-get update
* Install the MongoDB packages.
  - sudo apt-get install -y mongodb-org
  - mpngodb-org 패키지: A metapackage that will automatically install the four component packages, mongodb-org-server, mongodb-org-mongos, mongodb-org-shell, and mongodb-org-tools
  - The MongoDB instance stores its data files in /var/lib/mongodb and its log files in /var/log/mongodb by default, and runs using the mongodb user account. You can specify alternate log and data file directories in /etc/mongod.conf.
* Start MongoDB
  - sudo service mongod start
* Verify that MongoDB has started successfully
  - Verify that the mongod process has started successfully by checking the contents of the log file at /var/log/mongodb/mongod.log for a line reading
  - [initandlisten] waiting for connections on port <port>
  - where <port> is the port configured in /etc/mongod.conf, 27017 by default.
* Stop MongoDB
  - sudo service mongod stop

### 2.3. EC2(AWS 서버)에 mongodb  설정 방법 (ubuntu 기반)

* AWS Management Console -> EC2 -> Security Groups -> EC2's Security Group -> Add Custom TCP Rule, 27017, AnyWhere

* 외부 접속 허용
  - sudo vi /etc/mongod.conf
    - bindIp: 0.0.0.0   으로 변경
  - sudo service mongod restart

* 계정을 만들어야 함 (그렇지 않으면 외부에서 해당 포트/주소로 아무나 접속이 가능하기 때문)
  - EC2 에서 다음 명령 실행

<pre>
mongo
use admin
db.createUser( 
    {   user: "hyeyoung",
        pwd: "mongodb!@wrlove",
        roles: [ "userAdminAnyDatabase",
        "dbAdminAnyDatabase",
        "readWriteAnyDatabase"] 
    } 
)
</pre>

        - sudo vim /etc/mongod.conf
        - Add the following configurations

<pre>
security:
  authorization: enabled
</pre>

       - sudo service mongod restart

## 3. mongoDB 바로 다뤄보기

* mongodb의 데이터 구성
 - db, collection으로 구성
 - 데이터는 각 collection에 document 형식(python dictionary)으로 저장 됨
 - collection들의 논리적인 집합이 database

### 3.1. Robomongo 설치 (MongoDB 관리 GUI 툴) (실습)
* https://robomongo.org/download
  - 맥에서 처음 실행시 잘 안되면, 삭제 후, 다시 다운로드받아서 재설치하세요
* 실행
  - Click Create
    - In Connection
      - Name: AWS EC2 IP
      - Address: AWS EC2 IP (port는 27017 디폴트)
    - In Authentication
      - Database 이름: admin
      - User Name: 사용자 ID
      - Password: 사용자 암호
    - Click Test & Save button to check connection
  - Connection
    - Click Coonect buttion
  - Check connection
    - Right-click (server name) -> Show Log
  - Create DB
    - Right-click (Server) -> Create Database -> Add dave_db
  - Create Collection
    - Right-click (Collections(0) in created DB) -> Create Collection -> Add test
  - Insert Document
    - Right-click (test collection) -> Insert Document -> Add the following JSON(BSON) Document -> Click Validate & Save buttons
    - Click View Documents in test collection
    - Check _id_ in indexes of test collection (인덱스 자동 생성)

<pre>
{
    "name"  : "Dave Lee",
    "age"   : 22,
    "major" : "CS"
}
{
    "name"  : "David Oh",
    "age"   : 24,
    "major" : "Japanense"
}
</pre>

<img src="https://www.fun-coding.org/00_Images/robomongo.png" /> 

  - Insert Document (different JSON fields) & Click View Documents in test collection

<pre>
{
    "name"     : "이종수",
    "age"      : 54,
    "minor"    : "CS",
    "nickname" : "wink"
}
</pre>

<img src="https://www.fun-coding.org/00_Images/robomongo_diff.png" />

### 3.2. Robomongo에서 커멘드 입력해보기 (실습)
 - Right-click (Server) -> Open Shell
 - show dbs - 전체 데이터베이스 열람
 - use [DB 이름] - 데이터베이스 선택 (없으면 데이터베이스 생성)
   * 예) use dave_db - dave_db 데이터베이스 선택
   
 
 - show collections - 선택된 데이터베이스의 콜렉션 열람
 - db.[Collection 이름].함수() 로 해당 컬렉션의 데이터 열람 및 조작
   * 예) db.test.find() - test 컬렉션에서 전체 Document 검색

* 데이터베이스 생성
  - use dave_db
* 데이터베이스 현황 확인
  - db 
  - db.stats()
* collection 생성 및 삭제
  - db.createCollection("employees", {capped:true, size:10000})
    - Right-click (server) -> Click Refresh -> Go to dave_db -> Go to Collections -> Check employees
    - capped:true 최초 제한된 크기로 생성된 공간에서만 데이터를 저장하는 설정 (고성능, 저장공간차면 기존 공간 재사용, 일정시간만 저장하는 로그에 적합)
  - db.employees.isCapped()
  - db.employees.drop()
  - db.createCollection("employees")  
  - db.employees.isCapped()
  - db.employees.drop()
  - db.createCollection("employees", {capped:true, size:10000})
* collection 확인
  - show collections
  - db.employees.stats()
* collection 이름 변경
  - db.employees.renameCollection("emp")
* collection 삭제
  - db.emp.drop()

### 3.3. SQL과 간단 비교 - mongodb collection 생성/변경
* collection 생성 (원하는 타임으로 데이터를 바로 넣으면 됨)
  - PRIMARY KEY를 위한 별도 컬럼 만들 필요 없음. 
  - mongodb는 collection에서 _id가 각 Document마다 자동생성되어 primary key 역햘을 함
  - 컬럼마다 데이터 타입을 정할 필요 없음 ("컬럼명": 컬럼값 이 기본 형태임)

<img src="https://www.fun-coding.org/00_Images/mongodb_create.png" /> 

* collection 구조 변경 (기존 Document에 컬럼 추가/삭제 필요없을 시는 새로운 Document에만 필요한 컬럼을 추가 또는 삭제해서 넣으면됨)
  - ALTER TABLE은 기본적으로 collection에서는 필요 없음 
  - 일부 기존 Document에도 컬럼과 컬럼값을 넣거나 삭제해야 한다면 다음과 같은 형태로는 가능함

* 기존 Document에도 컬럼과 컬럼값 추가시 
  - SQL: ALTER TABLE people ADD COLUMN join_date DATETIME
  - mongodb: db.people.updateMany({ }, { $set: { join_date: new Date() } })

* 기존 Document에도 컬럼과 컬럼값 삭제시 
  - SQL: ALTER TABLE people DROP COLUMN join_date
  - mongodb: db.people.updateMany({ }, { $unset: { "join_date": "" } })

## 4. mongoDB 데이터 입력/검색/수정/삭제 (CRUD)

### 4.1. Document 입력 - insertOne, insertMany
  - insertOne : 한개의 document 생성
  - insertMany : list of document 생성

### Document 입력 문법

<img src="https://www.fun-coding.org/00_Images/mongodb_insert_structure.png" /> 

### SQL INSERT 문법과 비교

<img src="https://www.fun-coding.org/00_Images/mongodb_insert.png" /> 

* insertOne 예제

<pre>
db.articles.insertOne(
     { subject: "coffee", author: "xyz", views: 50 }
)
</pre>

* insertMany 예제

<pre>
db.articles.insertMany(
   [
     { subject: "coffee", author: "xyz", views: 50 },
     { subject: "Coffee Shopping", author: "efg", views: 5 },
     { subject: "Baking a cake", author: "abc", views: 90  },
     { subject: "baking", author: "xyz", views: 100 },
     { subject: "Café Con Leche", author: "abc", views: 200 },
     { subject: "Сырники", author: "jkl", views: 80 },
     { subject: "coffee and cream", author: "efg", views: 10 },
     { subject: "Cafe con Leche", author: "xyz", views: 10 },
     { subject: "coffees", author: "xyz", views: 10 },
     { subject: "coffee1", author: "xyz", views: 10 }
   ]
)
</pre>

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
1. employees Collection 생성 {capped:true, size:100000} Capped Collection, size는 100000 으로 생성<br>
2. 다음 Document 데이터 넣기<br>
   - user_id 가 bcd001, age 가 45, status 가 A 인 Document<br>
   - user_id 가 bcd002, age 가 25, status 가 B 인 Document<br>
   - user_id 가 bcd003, age 가 50, status 가 A 인 Document<br>
   - user_id 가 bcd004, age 가 35, status 가 A 인 Document<br>
   - user_id 가 abc001, age 가 28, status 가 B 인 Document<br>
</div>

### 4.2. Document 읽기(검색) - findOne, find
  - findOne : 매칭되는 한개의 document 검색
  - find : 매칭되는 list of document 검색

### Document 읽기(검색) 문법

<img src="https://www.fun-coding.org/00_Images/mongodb_find_structure.png" /> 

- find()/findOne 명령과 - SQL 문 비교

<pre>
db.people.find() - SELECT * FROM people
db.people.find({ }, { user_id: 1, status: 1 }) - SELECT _id, user_id, status FROM people
db.people.find({ },{ user_id: 1, status: 1, _id: 0 }) - SELECT user_id, status FROM people
db.people.find({ status: "A" }) - SELECT * FROM people WHERE status = "A"
db.people.find({ status: "A", age: 50 }) - SELECT * FROM people WHERE status = "A" AND age = 50
db.people.find({ $or: [ { status: "A" } , { age: 50 } ] }) - SELECT * FROM people WHERE status = "A" OR age = 50
</pre>

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
1. employees Collection 에서 user_id 가 bcd002 인 Document의 user_id, age, status, _id 출력<br>
2. employees Collection 에서 user_id 가 bcd003 인 Document의 user_id, age, status 출력<br>
3. employees Collection 에서 user_id 가 bcd004 이거나, age가 28인 Document 의 모든 필드 출력
</div>

### 비교 문법
<pre>
$eq     =    Matches values that are equal to a specified value.
$gt     >    Matches values that are greater than a specified value.
$gte    >=   Matches values that are greater than or equal to a specified value.
$in          Matches any of the values specified in an array.
$lt     <    Matches values that are less than a specified value.
$lte    <=   Matches values that are less than or equal to a specified value.
$ne     !=   Matches all values that are not equal to a specified value.
$nin         Matches none of the values specified in an array.
</pre>

### 비교 문법 코드 예제
<pre>
db.people.find({ age: { $gt: 25 } }) - SELECT * FROM people WHERE age > 25
db.people.find({ age: { $lt: 25 } }) - SELECT * FROM people WHERE age < 25
db.people.find({ age: { $gt: 25, $lte: 50 } }) - SELECT * FROM people WHERE age > 25 AND age <= 50
db.people.find( { age: { $nin: [ 5, 15 ] } } )) - SELECT * FROM people WHERE age = 5 or age = 15
db.people.find( { user_id: /bc/ } )
db.people.find( { user_id: { $regex: /bc/ } } )
                                                  - SELECT * FROM people WHERE user_id like "%bc%"
db.people.find( { user_id: /^bc/ } )
db.people.find( { user_id: { $regex: /^bc/ } } )
                                                  - SELECT * FROM people WHERE user_id like "bc%"
db.people.find( { status: "A" } ).sort( { user_id: 1 } ) - SELECT * FROM people WHERE status = "A" ORDER BY user_id ASC 
db.people.find( { status: "A" } ).sort( { user_id: -1 } ) - SELECT * FROM people WHERE status = "A" ORDER BY user_id DESC
db.people.count()
db.people.find().count()
                                                  - SELECT COUNT(*) FROM people
db.people.count( { user_id: { $exists: true } } )
db.people.find( { user_id: { $exists: true } } ).count()
                                                  - SELECT COUNT(user_id) FROM people
db.people.count( { age: { $gt: 30 } } )
db.people.find( { age: { $gt: 30 } } ).count()
                                                  - SELECT COUNT(*) FROM people WHERE age > 30
db.people.distinct( "status" ) - SELECT DISTINCT(status) FROM people
db.people.findOne()
db.people.find().limit(1)
                                                  - SELECT * FROM people LIMIT 1
</pre>

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
1. 다음 Document 데이터 넣기<br>
   - age 가 20 보다 큰 Document 의 user_id 만 출력하기<br>
   - age 가 50 이고 status 가 A 인 Document 의 user_id 만 출력하기<br>
   - age 가 60 보다 작은 Document 의 user_id 와 age 출력하기<br>
   - user_id 종류 출력하기<br>
   - user_id 가 bcd 로 시작하는 전체 Document 출력하기
</div>

### Document 수정 - updateOne, updateMany
  - updateOne - 매칭되는 한개의 document 업데이트
  - updateMany - 매칭되는 list of document 업데이트

### 4.3. Document 수정 문법

<img src="https://www.fun-coding.org/00_Images/mongodb_update_structure.png" />

- $set: field 값 설정
- $inc: field 값을 증가시키거나, 감소시킴
  - 예: $inc: { age: 2 } - age 값을 본래의 값에서 2 증가

### Document 수정 코드 예제

<pre>
- db.people.updateMany( { age: { $gt: 25 } }, { $set: { status: "C" } } )
- SQL 변환하면, 
  - UPDATE people SET status = "C" WHERE age > 25
- 한 Document만 수정하려면 updateOne을 사용함
- db.people.updateOne( { age: { $gt: 25 } }, { $set: { status: "C" } } )
- db.people.updateMany( { status: "A" } , { $inc: { age: 3 } } )
- SQL 변환하면,
  - UPDATE people SET age = age + 3 WHERE status = "A"
</pre>

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
1. 다음 Document 데이터 수정하기<br>
   - age 가 40 보다 큰 Document 의 status 를 B 로 변환하기
</div>

### 4.4. Document 삭제 - removeOne, removeMany
  - removeOne - 매칭되는 한개의 document 삭제
  - removeMany - 매칭되는 list of document 삭제


### Document 삭제 문법

<img src="https://www.fun-coding.org/00_Images/mongodb_delete_structure.png" /> 

- db.people.deleteMany( { status: "D" } )
- SQL로 변환하면,
  - DELETE FROM people WHERE status = "D"
- db.people.deleteMany({})
- SQL로 변환하면,
  - DELETE FROM people  

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
1. 다음 Document 데이터 삭제하기<br>
   - age 가 30 보다 작은 Document 삭제하기
</div>

### 참고: mongo shell
 - 로컬에서 서버가 돌아갈 경우,
   - mongo
 - 원격 서버에 접속할 경우 
   - mongo --host 'host_address' --port 'port'
   - 예) mongo --host 192.10.21.3 --port 27017


## 5. 파이썬으로 mongoDB 제어하기 - pymongo 라이브러리
 - mongodb python module
 - https://api.mongodb.com/python/current/
 - pip install pymongo

### 참고
1. pymongo 라이브러리 import
2. mongodb 접속 (주소)
3. 내가 사용할 database, collection 생성 또는 선택
4. 해당 database의 collection에 CRUD 명령하는 방법

### mongodb with EC2 Connection via using pymongo (코드 실행 실습)

In [ ]:
import pymongo

ImportError: No module named pymongo

### 5.1.  연결하기

In [ ]:
import pymongo
conn = pymongo.MongoClient()

ImportError: No module named pymongo

In [ ]:
# connection = pymongo.MongoClient(mongo_server, 27017)
connection = pymongo.MongoClient('mongodb://13.209.140.30')

NameError: name 'pymongo' is not defined

### 5.2. test Database 사용하기 (없으면 만들어진다.)

In [ ]:
knowledge = conn.knowledge

In [ ]:
mydb_test = conn["test"]   # 이렇게도 가능하다.

In [ ]:
print(knowledge)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'knowledge')


In [ ]:
print(dir(mydb_test))

['_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_Database__client', '_Database__incoming_copying_manipulators', '_Database__incoming_manipulators', '_Database__name', '_Database__outgoing_copying_manipulators', '_Database__outgoing_manipulators', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply_incoming_copying_manipulators', '_apply_incoming_manipulators', '_command', '_create_or_update_user', '_default_role', '_fix_incoming', '_fix_outgoing', '_list_collections', '_read_preference_for', '_write_concern_for', 'add_son_manipulator', 'add_user', 'auth

In [ ]:
print(knowledge.name)

knowledge


### 5.3.  test_collection 이라는 collection 사용하기 (없으면 만들어진다.)

In [ ]:
knowledge_it = knowledge.it

In [ ]:
knowledge_it = knowledge["it"]

In [ ]:
knowledge_it

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'knowledge'), 'it')

### 5.4.  Document INSERT 하기 ( insert_one() 과 insert_many() )

* insert_one()
  - mongodb shell 명령어: insertOne()

In [ ]:
post = {"author": "Mike", "text": "My first blog post!", "tags": ["mongodb", "python", "pymongo"] }

In [ ]:
knowledge_it.insert_one(post)

In [ ]:
knowledge_it.insert_one( { "author":"Dave Lee", "age":45 } )

* insert_many()

In [ ]:
knowledge_it.insert_many(
    [
        { "author":"Dave Ahn", "age":25 },
        { "author":"Dave", "age":35 }
    ]
)

* Document INSERT 하면, _id (primary key)를 확인하는 방법

In [ ]:
post = {"author": "Dave", "text": "My first blog post!"}

In [ ]:
post_id = knowledge_it.insert_one(post)

In [ ]:
post_id

In [ ]:
post_id.inserted_id

ObjectId('5d32a3abc92b6508c3f5d306')

* estimated_document_count() 메서드는 컬렉션 객체와 함께 쓰여서 총 Document 수를 알려줌
  - count_documents({})
  - count() 함수는 최신 pymongo 라이브러리에서는 사용 권장되지 않음

In [ ]:
knowledge_it.count_documents({})    

6

* list와 dictionary 를 활용하여 insert 하기

In [ ]:
# 리스트, 객체 삽입 가능
knowledge_it.insert_one({'title' : '암살', 'castings' : ['이정재', '전지현', '하정우']})
knowledge_it.insert_one(
    {
        'title' : '실미도', 
        'castings' : ['설경구', '안성기'], 
        'datetime' : 
        {
            'year' : '2003', 
            'month' : 3,
            'val' : 
            {
                'a' :
                {
                    'b' : 1
                }
            }
        }
    }
)

In [ ]:
data = list()
data.append({'name' : 'aaron', 'age' : 20})
data.append({'name' : 'bob', 'age' : 30})
data.append({'name' : 'cathy', 'age' : 25})
data.append({'name' : 'david', 'age' : 27})
data.append({'name' : 'erick', 'age' : 28})
data.append({'name' : 'fox', 'age' : 32})
data.append({'name' : 'hmm'})

knowledge_it.insert_many(data)

In [ ]:
knowledge_it.estimated_document_count()

17

### 5.5.  Document 검색 하기(읽기) ( find_one() 과 find() )

* find_one() 메서드 : 가장 빨리 검색되는 하나 검색하기

In [ ]:
knowledge_it.find_one()

{'_id': ObjectId('5d329c5fc92b6508c3f5d300'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo']}

In [ ]:
dave = knowledge_it.find_one( {"author":"Dave"} )
dave

{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave', 'age': 35}

* find_one( 안에 조건을 넣을 때는 사전 형식으로 해야 합니다. { 키:값 } )

* find() 메서드 : 검색되는 모든 Document 읽어오기

In [ ]:
docs = knowledge_it.find()

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5d329c5fc92b6508c3f5d300'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5d329d62c92b6508c3f5d301'), 'author': 'Dave Lee', 'age': 45}
{'_id': ObjectId('5d329da9c92b6508c3f5d302'), 'author': 'Dave Lee', 'age': 45}
{'_id': ObjectId('5d329de7c92b6508c3f5d303'), 'author': 'Dave Ahn', 'age': 25}
{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave', 'age': 35}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave', 'text': 'My first blog post!'}
{'_id': ObjectId('5d32a6fdc92b6508c3f5d307'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5d32a6fdc92b6508c3f5d308'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5d32a7acc92b6508c3f5d309'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5d32a7acc92b6508c3f5d30a'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year

In [ ]:
docs = knowledge_it.find( {"author":"Dave"} )

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave', 'age': 35}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave', 'text': 'My first blog post!'}


* count_documents() 함수로 조건에 맞는 검색 데이터 갯수 알아내기

In [ ]:
knowledge_it.count_documents({"author": "Dave"})

2

* sort() 와 함께 쓰기

In [ ]:
for post in knowledge_it.find().sort("age"):
    print(post)

{'_id': ObjectId('5d329c5fc92b6508c3f5d300'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave', 'text': 'My first blog post!'}
{'_id': ObjectId('5d32a6fdc92b6508c3f5d307'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5d32a6fdc92b6508c3f5d308'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5d32a7acc92b6508c3f5d309'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5d32a7acc92b6508c3f5d30a'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5d32a7e7c92b6508c3f5d311'), 'name': 'hmm'}
{'_id': ObjectId('5d32a7e7c92b6508c3f5d30b'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5d329de7c92b6508c3f5d303'), 'author': 'Dave Ahn', 'age': 25}
{'_id': ObjectId('5d32a7e7c92b6508c3f5d30d'), 'nam

### 5.6. Document Update 하기 (update_one() 과 update_many())

* update_one() : 가장 먼저 검색되는 한 Document만 수정하기

In [ ]:
knowledge_it.find_one( {"author":"Dave"} )

{'_id': ObjectId('5d329de7c92b6508c3f5d304'),
 'author': 'Dave',
 'age': 40,
 'text': 'Hi Dave'}

In [ ]:
knowledge_it.update_one( { "author" : "Dave" }, 
    { "$set" : 
        { "text" : "Hi Dave" }
    }
)

In [ ]:
docs = knowledge_it.find( {"author":"Dave Lee"} )

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5d329d62c92b6508c3f5d301'), 'author': 'Dave Lee', 'age': 30}
{'_id': ObjectId('5d329da9c92b6508c3f5d302'), 'author': 'Dave Lee', 'age': 30}
{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave Lee', 'age': 30, 'text': 'Hi Dave'}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave Lee', 'text': 'My first blog post!', 'age': 30}


* update_many() : 조건에 맞는 모든 Document 수정하기

In [ ]:
knowledge_it.update_many( {"author": "Dave Lee"}, {"$set": { "age": 30}})

### 5.7. Document 삭제 하기 (delete_one() 과 delete_many())

* delete_one() 메서드 : 가장 먼저 검색되는 한 Document만 삭제하기

In [ ]:
docs = knowledge_it.find( {"author":"Dave Lee"} )

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5d329d62c92b6508c3f5d301'), 'author': 'Dave Lee', 'age': 30}
{'_id': ObjectId('5d329da9c92b6508c3f5d302'), 'author': 'Dave Lee', 'age': 30}
{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave Lee', 'age': 30, 'text': 'Hi Dave'}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave Lee', 'text': 'My first blog post!', 'age': 30}


In [ ]:
knowledge_it.delete_one( {"author":"Dave Lee"} )

In [ ]:
docs = knowledge_it.find( {"author":"Dave Lee"} )

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5d329da9c92b6508c3f5d302'), 'author': 'Dave Lee', 'age': 30}
{'_id': ObjectId('5d329de7c92b6508c3f5d304'), 'author': 'Dave Lee', 'age': 30, 'text': 'Hi Dave'}
{'_id': ObjectId('5d32a3abc92b6508c3f5d306'), 'author': 'Dave Lee', 'text': 'My first blog post!', 'age': 30}


* delete_many() 메서드 : 조건에 맞는 모든 Document 삭제하기

In [ ]:
knowledge_it.delete_many( {"author":"Dave Lee"} )

In [ ]:
knowledge_it.count_documents( {"author":"Dave Lee"} )

0

In [ ]:
import pymongo
conn = pymongo.MongoClient()
books = conn.books
it_book = books.it_book

data = list()
for index in range(100):
    data.append({"author":"Dave Lee", "publisher":"fun-coding.org", "number": index })

In [ ]:
# CRUD - Create(Insert)
it_book.insert_many(data)

In [ ]:
# CRUD - Read
docs = it_book.find()
for doc in docs:
    print (doc)

{'_id': ObjectId('5d32bc4fc92b6508c3f5d313'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 0}
{'_id': ObjectId('5d32bc4fc92b6508c3f5d314'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 1}
{'_id': ObjectId('5d32bc4fc92b6508c3f5d315'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 2}
{'_id': ObjectId('5d32bc4fc92b6508c3f5d316'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 3}
{'_id': ObjectId('5d32bc4fc92b6508c3f5d317'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 4}
{'_id': ObjectId('5d32bc4fc92b6508c3f5d318'), 'author': 'Dave Lee', 'publisher': 'www.fun-coding.org', 'number': 5}


In [ ]:
# CRUD - Update
it_book.update_many( {}, { "$set": { "publisher":"www.fun-coding.org"} } )

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
number 가 6 이상(>=)인 doc 삭제하기
</div>

In [ ]:
# CRUD - Delete
it_book.delete_many( { "number": { "$gte": 6} } )

## 6. 크롤링과 함께 쓰는 mongoDB 예제

In [ ]:
import requests
import re
import datetime
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo

In [ ]:
username = 'davelee'
password = 'happy91'
connection = pymongo.MongoClient('mongodb://%s:%s@www.funcoding.xyz' % (username, password))

In [ ]:
mongodb = connection.cine21

In [ ]:
actor_collection = mongodb.actor_collection

In [ ]:
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

마동석
진선규
윤계상
조재윤
최귀화
허성태
임형준
조우진
홍기준
송영창
조진웅
허동원
김윤석
이병헌
박해일
정인기
고수
예정화
박희순
박지환
이하늬
이동휘
이다윗
나문희
김소진
이제훈
정연주
엄지성
신하균
김법래
염혜란
이상희
민경진
손숙
박철민
유순웅
김구택
박지일
송상은
윤병희
최민식
현빈
최종률
김래원
이지훈
도경수
박신혜
유지태
정재진
박성웅
문창길
박해준
김해숙
하준
조한철
이선균
배성우
김혜수
김중기
성동일
김일웅
심규혁
이수경


### cine21 인물 랭킹 알아내기
  - http://www.cine21.com/rank/person/
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2017-09
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [ ]:
cine21_url = 'http://www.cine21.com/rank/person/content'

In [ ]:
month = "2017-10"

In [ ]:
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = month
conditions['gender'] = 'all'
conditions['page'] = 1

In [ ]:
response = requests.post(cine21_url, data = conditions)

In [ ]:
response

<Response [200]>

In [ ]:
response.content

In [ ]:
soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

In [ ]:
soup

In [ ]:
actors = soup.select('li.people_li div.name')

In [ ]:
actors

[<div class="name"><a href="/db/person/info/?person_id=64614">마동석(3편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=82350">진선규(2편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=19889">윤계상(3편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=84000">조재윤(2편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=102264">최귀화(3편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=27131">허성태(3편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=34884">임형준(2편)</a></div>]

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
actors 리스트에서 배우 이름만 추출하기 예) 마동석(3편)  --> 마동석
</div>

In [ ]:
import re
for actor in actors:
    print(re.sub("\(\w+\)", "", actor.text))

마동석
진선규
윤계상
조재윤
최귀화
허성태
임형준


### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

### 크롤링해서, embedded document 로 각 배우별 상세 정보를 통째로 만들고, document 컬럼에 넣기

In [ ]:
actor_detail_info = list()

for actor in actors:

    actor_info_dict = dict()
    
    actor_info = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
    
    response_actor = requests.get(actor_info)
    
    soup_actor = BeautifulSoup(response_actor.content.decode('utf-8'), 'html.parser')
    
    actor_datas = soup_actor.select('ul.default_info')
    for actor_data in soup_actor.select('ul.default_info'):
        for actor_item in actor_data.select('li'):
            actor_item_text = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_text = re.sub('<.+?>', '', actor_item_text)
            actor_info_dict[actor_item.select_one('span.tit').text] = actor_item_text.strip()
    actor_detail_info.append(actor_info_dict)
print(actor_detail_info)

[{'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}, {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}, {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}, {'직업': '배우', '생년월일': '1974-09-15', '성별': '남', '홈페이지': 'https://www.instagram.com/jojaeyun/'}, {'직업': '배우', '생년월일': '1978-03-03', '성별': '남', '신장/체중': '181cm, 72kg'}, {'직업': '배우', '성별': '남'}, {'직업': '배우', '생년월일': '1974-05-10', '성별': '남', '신장/체중': '177cm, 71kg', '학교': '서울예술대학 연극과', '취미': '스쿼시, 피아노', '특기': '작곡, 노래', '소속사': '나무액터스'}]


In [ ]:
actor_info_dict = dict()

actor_info = 'http://www.cine21.com/db/person/info/?person_id=19889'

response_actor = requests.get(actor_info)

soup_actor = BeautifulSoup(response_actor.content.decode('utf-8'), 'html.parser')

actor_datas = soup_actor.select('ul.default_info')
for actor_data in soup_actor.select('ul.default_info'):
    for actor_item in actor_data.select('li'):
        actor_item_text = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_text = re.sub('<.+?>', '', actor_item_text)
        actor_info_dict[actor_item.select_one('span.tit').text] = actor_item_text.strip()
print(actor_info_dict)

{'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}


In [ ]:
actor_info_dict = dict()

actor_info = 'http://www.cine21.com/db/person/info/?person_id=19889'

response_actor = requests.get(actor_info)

soup_actor = BeautifulSoup(response_actor.content.decode('utf-8'), 'html.parser')

In [ ]:
soup_actor

In [ ]:
actor_datas = soup_actor.select('ul.default_info')

In [ ]:
actor_datas

[<ul class="default_info">
 <li><span class="tit">다른 이름</span>지오디;god</li>
 <li><span class="tit">직업</span>배우</li>
 <li><span class="tit">생년월일</span>1978-12-20</li>
 <li><span class="tit">성별</span>남</li>
 <li><span class="tit">홈페이지</span>
 <a href="https://www.facebook.com/saram.yoonkyesang" target="_blank">https://www.facebook.com/saram.yoonkyesang</a><br/>
 <a href="https://www.instagram.com/kyesang78/" target="_blank">https://www.instagram.com/kyesang78/</a><br/>
 </li>
 <li><span class="tit">신장/체중</span>182cm, 62kg</li>
 <li><span class="tit">학교</span>경희대학교 포스트모던학과 휴학</li>
 <li><span class="tit">취미</span>컴퓨터게임, 스노우보드</li>
 <li><span class="tit">특기</span>표정연기, 춤추기</li>
 </ul>]

In [ ]:
for actor_data in soup_actor.select('ul.default_info'):
    for actor_item in actor_data.select('li'):
        actor_item_text = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_text = re.sub('<.+?>', '', actor_item_text)
        actor_info_dict[actor_item.select_one('span.tit').text] = actor_item_text.strip()

In [ ]:
print(actor_info_dict)

{'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}


### 흥행지수 뽑기

In [ ]:
actor_rates = soup.select('li.people_li ul.num_info strong')

In [ ]:
for actor_rate in actor_rates:
    print(actor_rate.text)

59,361
42,733
40,181
37,957
35,754
33,659
33,487


### 흥행지수 숫자로 만들어주기

In [ ]:
actor_rate_list = list()

In [ ]:
for actor_rate in actor_rates:
    actor_rate_list.append(int(actor_rate.text.replace(",","")))  # int() 로 해주지 않으면, 문자열로 됩니다.

In [ ]:
actor_rate_list

[59361, 42733, 40181, 37957, 35754, 33659, 33487]

In [ ]:
actor_list = list()

In [ ]:
for actor in actors:
    actor_list.append(re.sub("\(\w+\)", "", actor.text))

In [ ]:
actor_list

['마동석', '진선규', '윤계상', '조재윤', '최귀화', '허성태', '임형준']

### 각 배우별 출연 영화를 document에 저장하고 싶다.
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

* 어떻게? 다음과 같이 합니다.

In [ ]:
movie_list = list()

In [ ]:
movies = soup.select('li.people_li ul.mov_list')

In [ ]:
for movie in movies:
    actor_movie = list()
    movie_titles = movie.select('li a span')
    for movie_title in movie_titles:
        actor_movie.append(movie_title.text)
    movie_list.append(actor_movie)

In [ ]:
movie_list

[['범죄도시', '부라더', '부산행'],
 ['범죄도시', '남한산성'],
 ['범죄도시', '발레 교습소', '죽여주는 여자'],
 ['범죄도시', '역모 - 반란의 시대'],
 ['범죄도시', '택시운전사', '조작된 도시'],
 ['남한산성', '범죄도시', '부라더'],
 ['범죄도시', '이웃집 스타']]

* 이렇게 만든 리스트를 넣으면 됩니다.

### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

In [ ]:
for num, actor in enumerate(actor_list):
    actor_collection.insert_one(
        {"actor":actor_list[num], 
         "actor_details": actor_detail_info[num], 
         "actor_rate":actor_rate_list[num], 
         "date":month, 
         "movie_list":movie_list[num]})

In [ ]:
docs = actor_collection.find()

In [ ]:
for doc in docs:
    print(doc)

### collection 삭제하기

In [ ]:
actor_collection.drop()

In [ ]:
docs = actor_collection.find()

In [ ]:
for doc in docs:
    print(doc)

### Dictionary 타입으로 만들어서 한번에 insert_many() 로 데이터 입력하기

In [ ]:
actor_info = list()

In [ ]:
for num, actor in enumerate(actor_list):
    actor_info.append(
        {"actor":actor_list[num], 
         "actor_details": actor_detail_info[num], 
         "actor_rate":actor_rate_list[num], 
         "date":month, 
         "movie_list":movie_list[num]}
    )

In [ ]:
actor_info

In [ ]:
actor_collection.insert_many(actor_info)

In [ ]:
docs = actor_collection.find()

In [ ]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor': '마동석', 'actor_details': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}, 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행']}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor': '진선규', 'actor_details': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}, 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성']}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor': '윤계상', 'actor_details': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}, 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자']}
{'_id': ObjectId('5a0ae08481f6402f8ff056bf'), 'actor': '조재윤', 'actor_details': {'직업': '배우', '생년월일': 

### Update (컬럼명 변경 예제)

In [ ]:
actor_collection.update_many( {}, { "$rename": { "actor": "actor_name" } } )

In [ ]:
docs = actor_collection.find()
for doc in docs:
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_details': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}, 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석'}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_details': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}, 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규'}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_details': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}, 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상'}
{'_id': ObjectId('5a0ae08481f6402f8ff056bf'), 'actor_details': {'직업': '배우', '생년월일': '

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
actor_details 필드 이름을 actor_info 로 변경하기
</div>

In [ ]:
actor_collection.update_many( {}, { "$rename": { "actor_details": "actor_info" } } )

In [ ]:
docs = actor_collection.find()
for doc in docs:
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bf'), 'actor_rate': 37957, 'date': '2017-10', 'movie_l

* 컬렉션 객체 이름도 바꿀 수 있겠지요

In [ ]:
actor = actor_collection  

## 7. 예제로 이해하는 find 의 다양한 문법

### 7.1. sort
 - mysql의 order by에 해당
 - find로 데이터 열람 시, 정렬 조건 명시 가능

In [ ]:
# 기본적으로 오름차순
result = actor_collection.find().sort('actor_name')
for record in result:
    print(record)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056c2'), 'actor_rate': 33487, 'date': '2017-10', 'movie_list': ['범죄도시', '이웃집 스타'], 'actor_name': '임형준', 'actor_info': {'직업': '배우', '생년월일': '1974-05-10', '성별': '남', '신장/체중': '177cm, 71kg', '학교': '서울예술대학 연극과', '취미': '스쿼시, 피아노', '특기': '작곡, 노래', '소속사': '나무액터스'}

In [ ]:
result = actor_collection.find().sort('actor_name', pymongo.ASCENDING)
for record in result:
    print(record)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056c2'), 'actor_rate': 33487, 'date': '2017-10', 'movie_list': ['범죄도시', '이웃집 스타'], 'actor_name': '임형준', 'actor_info': {'직업': '배우', '생년월일': '1974-05-10', '성별': '남', '신장/체중': '177cm, 71kg', '학교': '서울예술대학 연극과', '취미': '스쿼시, 피아노', '특기': '작곡, 노래', '소속사': '나무액터스'}

In [ ]:
# 내림차순 명시
result = actor_collection.find().sort('actor_name', pymongo.DESCENDING)
for record in result:
    print(record)

{'_id': ObjectId('5a0ae08481f6402f8ff056c1'), 'actor_rate': 33659, 'date': '2017-10', 'movie_list': ['남한산성', '범죄도시', '부라더'], 'actor_name': '허성태', 'actor_info': {'직업': '배우', '성별': '남'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056c0'), 'actor_rate': 35754, 'date': '2017-10', 'movie_list': ['범죄도시', '택시운전사', '조작된 도시'], 'actor_name': '최귀화', 'actor_info': {'직업': '배우', '생년월일': '1978-03-03', '성별': '남', '신장/체중': '181cm, 72kg'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bf'), 'actor_rate': 37957, 'date': '2017-10', 'movie_list': ['범죄도시', '역모 - 반란의 시대'], 'actor_name': '조재윤', 'actor_info': {'직업': '배우', '생년월일': '1974-09-15', '성별': '남', '홈페이지': 'https://www.instagram.com/jojaeyun/'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056c2'), 'actor_rate': 33487, 'date': '2017-10', 'movie_list': ['범죄도시', '이웃집 스타'], 'actor

In [ ]:
# sort by multiple fields
result = actor_collection.find().sort([('actor_name', pymongo.ASCENDING),
                            ('actor_rate', pymongo.DESCENDING)])

for record in result:
    print(record)

NameError: name 'actor_collection' is not defined

### 7.2. 필드값이 존재하는 경우 검색 예제 (exists)
  - embedded document 의 경우 검색 예제 포함

In [ ]:
for doc in actor_collection.find({'actor_info.특기' : {'$exists' : False}}):
    print(doc)

{'_id': ObjectId('5a0995cd81f64002aac0bf1c'), 'actor_rate': 59190, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf1d'), 'actor_rate': 42900, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf1f'), 'actor_rate': 38068, 'date': '2017-10', 'movie_list': ['범죄도시', '역모 - 반란의 시대'], 'actor_name': '조재윤', 'actor_info': {'직업': '배우', '생년월일': '1974-09-15', '성별': '남', '홈페이지': 'https://www.instagram.com/jojaeyun/'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf20'), 'actor_rate': 35859, 'date': '2017-10', 'movie_list': ['범죄도시', '택시운전사', '조작된 도시'], 'actor_name': '최귀화', 'actor_info': {'직업': '배우', '생년월일': '1978-03-03', '성별': '남', '신장/체중': '181cm, 72kg'}}
{'_id': ObjectId('5a0

In [ ]:
for doc in actor_collection.find({'actor_info.생년월일' : {'$exists' : False}}):
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056c1'), 'actor_rate': 33659, 'date': '2017-10', 'movie_list': ['남한산성', '범죄도시', '부라더'], 'actor_name': '허성태', 'actor_info': {'직업': '배우', '성별': '남'}}


In [ ]:
for doc in actor_collection.find({'actor_info.홈페이지' : {'$exists' : True}}):
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bf'), 'actor_rate': 37957, 'date': '2017-10', 'movie_list': ['범죄도시', '역모 - 반란의 시대'], 'actor_name': '조재윤', 'actor_info': {'직업': '배우', '생년월일': '1974-09-15', '성별': '남', '홈페이지': 'https://www.instagram.com/jojaeyun/'}}


<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
생년월일이 없는 doc의 actor_name 만 출력하기
</div>

In [ ]:
for doc in actor_collection.find({'actor_info' : {'$exists' : True}}):
    print(doc)

{'_id': ObjectId('5a0995cd81f64002aac0bf1c'), 'actor_rate': 59190, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf1d'), 'actor_rate': 42900, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf1e'), 'actor_rate': 40298, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0995cd81f64002aac0bf1f'), 'actor_rate': 38068, 'date': '2017-10', 'movie_l

### 7.3. 필드의 범위로 검색 예제
* gt, gte
 - gt : 크다, gte : 크거나 같다
* lt, lte
 - lt : 작다, lte : 작거나 같다

In [ ]:
for doc in actor_collection.find({'actor_rate' : {'$gte' : 40000}}):
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}


In [ ]:
for doc in actor_collection.find(
                    {'actor_rate' : {'$lte' : 70000}, 
                     'actor_name' : '설경구' }
                    ):
    print(doc)

<div class="alert alert-block alert-success">
<font color="blue" size="4em">실습</font><br>
actor_rate 가 <= 35000 인 doc의 배우 이름과 배우가 출연한 영화 리스트 출력하기
</div>

In [ ]:
for doc in actor_collection.find( { 'actor_rate' : {'$lte' : 35000} }, { 'actor_name': 1, 'movie_list': 1, '_id':0 } ):
    print(doc)

{'movie_list': ['남한산성', '범죄도시', '부라더'], 'actor_name': '허성태'}
{'movie_list': ['범죄도시', '이웃집 스타'], 'actor_name': '임형준'}


### 7.4. or

#### test_insert 컬렉션으로 테스트를 해보겠습니다. (age 컬럼 활용)

In [ ]:
username = 'davelee'
password = 'korea123'
connection = pymongo.MongoClient('mongodb://%s:%s@www.funcoding.xyz' % (username, password))
db = connection.test
test_collection = db.test_collection

In [ ]:
for result in test_collection.find():
    print(result)

{'_id': ObjectId('5a01b4ba81f6400b2d7c677a'), 'author': 'Dave', 'text': 'My second blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5a01b50ec607715c40efc075'), 'author': 'Dave', 'text': 'My second blog post!', 'tags': ['mongodb', 'python', 'pymongo']}
{'_id': ObjectId('5a01b539c607715c40efc076'), 'number': 0}
{'_id': ObjectId('5a01b539c607715c40efc077'), 'number': 1}
{'_id': ObjectId('5a01b539c607715c40efc078'), 'number': 2}
{'_id': ObjectId('5a01b539c607715c40efc079'), 'number': 3}
{'_id': ObjectId('5a01b539c607715c40efc07a'), 'number': 4}
{'_id': ObjectId('5a01b539c607715c40efc07b'), 'number': 5}
{'_id': ObjectId('5a01b54081f6400b2d7c677b'), 'number': 0}
{'_id': ObjectId('5a01b54081f6400b2d7c677c'), 'number': 1}
{'_id': ObjectId('5a01b54081f6400b2d7c677d'), 'number': 2}
{'_id': ObjectId('5a01b54081f6400b2d7c677e'), 'number': 3}
{'_id': ObjectId('5a01b54081f6400b2d7c677f'), 'number': 4}
{'_id': ObjectId('5a01b54081f6400b2d7c6780'), 'number': 5}
{'_id': ObjectId

In [ ]:
for doc in test_insert_collection.find():
    print(doc)

{'_id': ObjectId('5a01b540c607715c40efc080'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5a01b540c607715c40efc081'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('5a01b541c607715c40efc082'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a01b541c607715c40efc083'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a01b541c607715c40efc084'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a01b541c607715c40efc085'), 'name': 'david', 'age': 27}
{'_id': ObjectId('5a01b541c607715c40efc086'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('5a01b541c607715c40efc087'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('5a01b541c607715c40efc088'), 'name': 'hmm'}
{'_id': ObjectId('5a09890f81f64002aac0bf0b'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('5a09890f81f64002aac0bf0c'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1

In [ ]:
for doc in test_insert_collection.find({'$or' : [ {'age' : 27}, {'age' : 30} ] }):
    print(doc)

{'_id': ObjectId('5a01b541c607715c40efc083'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a01b541c607715c40efc085'), 'name': 'david', 'age': 27}
{'_id': ObjectId('5a09893f81f64002aac0bf0e'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a09893f81f64002aac0bf10'), 'name': 'david', 'age': 27}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ad'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a0ac9b681f6402f8ff056af'), 'name': 'david', 'age': 27}


In [ ]:
# name이 aaron이고, age가 20이거나 30인 doc
for doc in test_insert_collection.find({ 'name' : 'aaron', '$or' : [ { 'age' : {'$lte' : 20 } }, { 'age' : 30 } ] }):
    print(doc)

{'_id': ObjectId('5a01b541c607715c40efc082'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a09893f81f64002aac0bf0d'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ac'), 'name': 'aaron', 'age': 20}


In [ ]:
for doc in test_insert_collection.find({'$or' : [ {'age' : { '$gte' : 29 } }, { 'age' : { '$lte' : 25 } } ] }):
    print(doc)

{'_id': ObjectId('5a01b541c607715c40efc082'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a01b541c607715c40efc083'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a01b541c607715c40efc084'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a01b541c607715c40efc087'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('5a09893f81f64002aac0bf0d'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a09893f81f64002aac0bf0e'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a09893f81f64002aac0bf0f'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a09893f81f64002aac0bf12'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ac'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ad'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ae'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a0ac9b681f6402f8ff056b1'), 'name': 'fox', 'age': 32}


In [ ]:
# age가 29 이상이거나 25인 doc
for doc in test_insert_collection.find( {'$or' : [ {'age' : {'$gte' : 29 } }, { 'age' : 25 } ] } ):
    print(doc)

{'_id': ObjectId('5a01b541c607715c40efc083'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a01b541c607715c40efc084'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a01b541c607715c40efc087'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('5a09893f81f64002aac0bf0e'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a09893f81f64002aac0bf0f'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a09893f81f64002aac0bf12'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ad'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('5a0ac9b681f6402f8ff056ae'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('5a0ac9b681f6402f8ff056b1'), 'name': 'fox', 'age': 32}


* 조건: age >= 27 or name = 'fox'
* 출력: name, age

In [ ]:
for doc in test_insert_collection.find( {'$or' : [ {'age' : {'$gte' : 27 } }, { 'name' : 'fox' } ] }, {'_id':0} ):
    print(doc)

{'name': 'bob', 'age': 30}
{'name': 'david', 'age': 27}
{'name': 'erick', 'age': 28}
{'name': 'fox', 'age': 32}
{'name': 'bob', 'age': 30}
{'name': 'david', 'age': 27}
{'name': 'erick', 'age': 28}
{'name': 'fox', 'age': 32}
{'name': 'bob', 'age': 30}
{'name': 'david', 'age': 27}
{'name': 'erick', 'age': 28}
{'name': 'fox', 'age': 32}


In [ ]:
# age가 29 이상이거나 25인 doc
for doc in test_insert_collection.find({'$or' : [ {'age' : {'$gte' : 27}}, {'name' : 'fox'}]}, {'name':1, 'age':1, '_id':0}):
    print(doc)

{'name': 'bob', 'age': 30}
{'name': 'david', 'age': 27}
{'name': 'erick', 'age': 28}
{'name': 'fox', 'age': 32}
{'name': 'bob', 'age': 30}
{'name': 'david', 'age': 27}
{'name': 'erick', 'age': 28}
{'name': 'fox', 'age': 32}


### 7.5. nor
 - not or

In [ ]:
for doc in test_insert_collection.find({'$nor' : [{'age' : {'$gte' : 29}}, {'age' : 25}]}):
    print(doc)

{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a47'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a48'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a49'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4c'), 'name': 'david', 'age': 27}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4d'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4f'), 'name': 'hmm'}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a58'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a59'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5a'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5d'), 'name': 'david', 'age': 27}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5e'), 'name': 'erick', 'a

### 7.6. in, nin

In [ ]:
# age가 해당 리스트안에 존재하는 원소와 일치할 떄
for doc in test_insert_collection.find({'age' : {'$in' : [20, 21, 25, 27]}}):
    print(doc)

{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a49'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4b'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4c'), 'name': 'david', 'age': 27}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5a'), 'name': 'aaron', 'age': 20}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5c'), 'name': 'cathy', 'age': 25}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5d'), 'name': 'david', 'age': 27}


In [ ]:
# nin : not in
for doc in test_insert_collection.find({'age' : {'$nin' : [20, 21, 25, 27]}}):
    print(doc)

{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a47'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a48'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4a'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4d'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4e'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4f'), 'name': 'hmm'}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a58'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a59'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5b'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5e'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5f'), 'name': 'fox', 'age': 32}

### 7.7. skip, limit
 - skip(n) : 검색 결과 n개만큼 건너뜀
 - limit(n) : 검색 결과 n개로 제한

In [ ]:
for doc in test_insert_collection.find({'age' : {'$nin' : [20, 21, 25, 27]}}).skip(3):
    print(doc)

{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4d'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4e'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4f'), 'name': 'hmm'}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a58'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a59'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5b'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5e'), 'name': 'erick', 'age': 28}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a5f'), 'name': 'fox', 'age': 32}
{'_id': ObjectId('59eaaf3a71f3ec3a5a4f7a60'), 'name': 'hmm'}


In [ ]:
for doc in test_insert_collection.find({'age' : {'$nin' : [20, 21, 25, 27]}}).limit(3):
    print(doc)

{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a47'), 'title': '암살', 'castings': ['이정재', '전지현', '하정우']}
{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a48'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4a'), 'name': 'bob', 'age': 30}


In [ ]:
# skip limit 동시 사용 가능
for doc in test_insert_collection.find({'age' : {'$nin' : [20, 21, 25, 27]}}).skip(1).limit(3):
    print(doc)

{'_id': ObjectId('59ea1a0471f3ec3a5a4f7a48'), 'title': '실미도', 'castings': ['설경구', '안성기'], 'datetime': {'year': '2003', 'month': 3, 'val': {'a': {'b': 1}}}}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4a'), 'name': 'bob', 'age': 30}
{'_id': ObjectId('59ea1a0571f3ec3a5a4f7a4d'), 'name': 'erick', 'age': 28}


### 7.7. list 검색

In [ ]:
import pymongo

username = 'davelee'
password = 'korea123'
connection = pymongo.MongoClient('mongodb://%s:%s@www.funcoding.xyz' % (username, password))
mongodb = connection.cine21
actor_collection = mongodb.actor_collection

In [ ]:
# 영화 리스트에 범죄도시를 포함하는 경우
for doc in actor_collection.find({'movie_list' : '범죄도시'}):
    print(doc['actor_name'])

마동석
진선규
윤계상
조재윤
최귀화
허성태
임형준


In [ ]:
# 영화 리스트에 범죄도시 또는 수상한 그녀를 포함하는 경우
for doc in actor_collection.find({'$or' : [{'movie_list' : '범죄도시'}, {'movie_list' : '수상한 그녀'}]}):
    print(doc['actor_name'])

마동석
진선규
윤계상
조재윤
최귀화
허성태
임형준


In [ ]:
# 영화 리스트에 범죄도시와 택시운전사를 포함하는 경우
for doc in actor_collection.find({'movie_list' : '범죄도시', 'movie_list' : '택시운전사'}):
    print(doc['actor_name'])

최귀화


* 리스트로 비교하기 위해서는 순서와 값이 정확히 일치해야 함

In [ ]:
# 결과 없음
for doc in actor_collection.find({'movie_list' : ['부라더', '범죄도시']}):
    print(doc['actor_name'])

In [ ]:
for doc in actor_collection.find({'movie_list' : ['범죄도시', '남한산성']}):
    print(doc['actor_name'])

진선규


* 순서에 관계 없이 찾고자 하는 경우

In [ ]:
# 영화 리스트에 부라더와 범죄도시를 순서와 관계없이 포함하는 경우
for doc in actor_collection.find({'movie_list' : {'$all' :  ['부라더', '범죄도시'] }} ):
    print(doc['actor_name'])

마동석
허성태


In [ ]:
# 영화 리스트에 부라더와 범죄도시를 순서와 관계없이 포함하는 경우
for doc in actor_collection.find({'movie_list' : {'$all' :  ['택시운전사', '범죄도시'] }} ):
    print(doc['actor_name'])

최귀화


In [ ]:
# 영화 리스트에 수상한 그녀를 순서와 관계없이 포함하는 경우
for doc in actor_collection.find({'movie_list' : {'$all' :  ['수상한 그녀'] }} ):
    print(doc['actor_name'])

* elemMatch
 - 적어도 한개 이상의 원소가 복수개의 조건을 동시에 만족하는 경우
 - 조건이 한개인 경우는 사용하지 않음

In [ ]:
score = mongodb.test.score

In [ ]:
for doc in score.find():
    print(doc)

{'_id': ObjectId('5a099d9f81f64002aac0bf25'), 'results': [82, 85, 88]}
{'_id': ObjectId('5a099d9f81f64002aac0bf26'), 'results': [75, 88, 89]}
{'_id': ObjectId('5a0a7e6681f6401787fdff7e'), 'results': [82, 85, 88]}
{'_id': ObjectId('5a0a7e6681f6401787fdff7f'), 'results': [75, 88, 91]}


In [ ]:
score.insert_many([
{ 'results': [ 82, 85, 88 ] },
{ 'results': [ 75, 88, 91 ] }])

In [ ]:
# 각 조건별로 하나라도 만족하는 원소가 있을 경우
for doc in score.find({'results' : { '$gte': 90, '$lt': 85 }}):
    print(doc)

{'_id': ObjectId('5a0a7e6681f6401787fdff7f'), 'results': [75, 88, 91]}


In [ ]:
# 적어도 한개의 원소(82) 가 모든 조건을 만족하는 경우 (80 <= 82 <= 85)
for doc in score.find({'results' :  {'$elemMatch' : { '$gte': 80, '$lt': 85 }}}):
    print(doc)

{'_id': ObjectId('5a099d9f81f64002aac0bf25'), 'results': [82, 85, 88]}
{'_id': ObjectId('5a0a7e6681f6401787fdff7e'), 'results': [82, 85, 88]}


* 인덱스를 특정하여 검색하기

In [ ]:
# movie_list의 1번째 원소가 범죄도시인 것 검색
for doc in actor_collection.find({'movie_list.0' : '범죄도시'} ):
    print(doc['actor_name'])

마동석
진선규
윤계상
조재윤
최귀화
임형준


In [ ]:
# movie_list의 2번째 원소가 범죄도시인 것 검색
for doc in actor_collection.find({'movie_list.1' : '범죄도시'} ):
    print(doc['actor_name'])

허성태


* 리스트의 원소의 개수를 기준으로 검색 (정확한 개수로만 검색 가능)

In [ ]:
for doc in actor_collection.find({'movie_list' : {'$size' : 3}} ):
    print(doc['actor_name'])

마동석
윤계상
최귀화
허성태


### 7.8. Update 예제

In [ ]:
result = actor_collection.update_one(
    {'actor_name' : '마동석'}, 
    {
        '$set' : 
        {
            'actor_info.생년월일' : '1971-03-03', 
            'actor_info.홈페이지' : 'https://www.instagram.com/madongseok'
        }
    }
)

In [ ]:
result.modified_count

1

In [ ]:
# update_many로 매칭되는 모든 doc 업데이트
result = actor_collection.update_many({'actor_info.성별' : '남'}, {'$set' : {'actor_info.성별' : '남자'}})

In [ ]:
result.modified_count

7

In [ ]:
# update_many로 매칭되는 모든 doc 업데이트
result = actor_collection.update_many({'actor_info.성별' : '여'}, {'$set' : {'actor_info.성별' : '여자'}})

In [ ]:
result.modified_count

0

In [ ]:
for doc in actor_collection.find():
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-03', '성별': '남자', '홈페이지': 'https://www.instagram.com/madongseok'}, 'actor_details': {'생년월일': '1971-03-03', '홈페이지': 'https://www.instagram.com/madongseok'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남자'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남자', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff0

* upsert option
 - update method의 upsert parameter가 True로 전달하면, 매칭되는 document를 찾지 못한 경우, insert 수행

In [ ]:
result = actor_collection.update_many({'actor_name' : '오달수'}, 
                           {'$set' : {'actor_name' : '오달수', 'actor_rate' : 42230}}, 
                           upsert=True)

In [ ]:
result.modified_count

0

In [ ]:
dir(result)

['_UpdateResult__acknowledged',
 '_UpdateResult__raw_result',
 '_WriteResult__acknowledged',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_raise_if_unacknowledged',
 'acknowledged',
 'matched_count',
 'modified_count',
 'raw_result',
 'upserted_id']

In [ ]:
# 새로 삽입된 객체의 아이디
result.upserted_id

ObjectId('5a0ae84ce8393b01d3212e0a')

In [ ]:
result = actor_collection.update_one({'actor_name' : '류승룡'}, 
                           {'$set' : {'actor_name' : '류승룡', 'actor_rate' : 42230}}, 
                           upsert=True)

In [ ]:
result.modified_count

0

In [ ]:
# 새로 삽입된 객체의 아이디
result.upserted_id

ObjectId('5a0ae85be8393b01d3212e1d')

In [ ]:
result = actor_collection.update_one({'actor_name' : '유해진'}, 
                           {'$set' : {'actor_name' : '유해진', 'actor_rate' : 42230}}, 
                           upsert=True)

In [ ]:
# 새로 삽입된 객체의 아이디
result.upserted_id

ObjectId('5a0ae861e8393b01d3212e23')

In [ ]:
for doc in actor_collection.find():
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-03', '성별': '남자', '홈페이지': 'https://www.instagram.com/madongseok'}, 'actor_details': {'생년월일': '1971-03-03', '홈페이지': 'https://www.instagram.com/madongseok'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남자'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남자', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff0

### 7.9. Delete 예제 

In [ ]:
result = actor_collection.delete_one({'actor_name' : '오달수'})
print (result.deleted_count)

result = actor_collection.delete_many({'name' : '오바마'})
print (result.deleted_count)

1
0


In [ ]:
result = actor_collection.delete_one({'actor_name' : '오달수'})

In [ ]:
result = actor_collection.delete_many({'actor_rate' : 42230})

In [ ]:
for doc in actor_collection.find():
    print(doc)

{'_id': ObjectId('5a0ae08481f6402f8ff056bc'), 'actor_rate': 59361, 'date': '2017-10', 'movie_list': ['범죄도시', '부라더', '부산행'], 'actor_name': '마동석', 'actor_info': {'직업': '배우', '생년월일': '1971-03-03', '성별': '남자', '홈페이지': 'https://www.instagram.com/madongseok'}, 'actor_details': {'생년월일': '1971-03-03', '홈페이지': 'https://www.instagram.com/madongseok'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056bd'), 'actor_rate': 42733, 'date': '2017-10', 'movie_list': ['범죄도시', '남한산성'], 'actor_name': '진선규', 'actor_info': {'직업': '배우', '생년월일': '1977-09-13', '성별': '남자'}}
{'_id': ObjectId('5a0ae08481f6402f8ff056be'), 'actor_rate': 40181, 'date': '2017-10', 'movie_list': ['범죄도시', '발레 교습소', '죽여주는 여자'], 'actor_name': '윤계상', 'actor_info': {'다른 이름': '지오디;god', '직업': '배우', '생년월일': '1978-12-20', '성별': '남자', '홈페이지': 'https://www.facebook.com/saram.yoonkyesang\nhttps://www.instagram.com/kyesang78/', '신장/체중': '182cm, 62kg', '학교': '경희대학교 포스트모던학과 휴학', '취미': '컴퓨터게임, 스노우보드', '특기': '표정연기, 춤추기'}}
{'_id': ObjectId('5a0ae08481f6402f8ff0

## 8. mongoDB 인덱스(INDEX)
 - https://docs.mongodb.com/manual/indexes/
 - SQL index와 개념적으로 동일
 - 즉, 검색을 더 빠르게 수행하고자 만드는 추가적인 data structure
   - index가 없으면 collection scan (컬렉션의 데이터를 하나하나 조회하는 방식) 으로 검색을 하게 됨

* Document의 필드들에 index 를 걸면, 데이터의 설정한 키 값을 가지고 document들을 가리키는 포인터 값으로 이뤄진 B-Tree 데이터 구조를 만듬
  - B-Tree : Balanced Binary search Tree, Binary Search (이진 검색) 으로 쿼리 속도를 검색 속도 개선

### 8.1. 기본 인덱스 _id
* 모든 MongoDB의 컬렉션은 기본적으로 _id 필드에 인덱스가 존재
* mongodb는 _id 를 기반으로 기본 인덱스 생성

### 8.2. Single(단일) 필드 인덱스
* _id 인덱스 외에도, 사용자가 지정 할 수 있는 단일 필드 인덱스
* 기본 사용 문법(mongodb CLI): db.COLLECTION.createIndex( { 'field': 1 } ) 또는 db.COLLECTION.createIndex( { 'field': -1 } )
  - 키의 값에는 1, -1 둘중의 하나가 올 수 있음
  - 1 : 순방향(ASCENDING), -1 : 역방향(DESCENDING)

### 8.3. Compound (복합)  필드 인덱스
* 두개 이상의 필드를 사용하는 인덱스를 복합 인덱스라고 부름
* 기본 사용 문법(mongodb CLI): db.COLLECTION.createIndex( { 'field1': 1, 'field2': -1} )
* 인덱스 방향이 성능에 영향을 미침
  - createIndex({a : 1, b : -1})로 생성한 경우
  - db.x.find({a : 1, b : -1}), db.x.find({a : -1, b : 1}) 의 쿼리만 효과를 봄
  - db.x.find({a : 1, b : 1}), db.x.find({a : -1, b : -1}) 의 쿼리는 효과 없음
* 복합 인덱스의 경우 필드의 개수가 31개를 넘을 수 없음

### 8.4. Text 인덱스
* 텍스트 관련 데이터를 효율적으로 쿼리하기 위한 인덱스
* 기본 사용 문법(mongodb CLI): db.COLLECTION.createIndex( { 'field' : 'text' } )

#### pymongo에서는 create_index() 메서드를 제공함

### 8.5. 코드로 작성해보면서 이해하는 인덱스

In [ ]:
import pymongo

In [ ]:
username = 'davelee'
password = 'korea123'
connection = pymongo.MongoClient('mongodb://%s:%s@www.funcoding.xyz' % (username, password))
db = connection.test_index_db

In [ ]:
# test_index_db 에는 어떤 컬렉션이 있을까?
db.collection_names()

['articles']

In [ ]:
# test data 삽입
db.articles.insert_many(
    [
        { "name": "Java Hut", "description": "Coffee and cakes", "ranking": 1 },
        { "name": "Burger Buns", "description": "Java hamburgers", "ranking": 2 },
        { "name": "Coffee Shop", "description": "Just coffee", "ranking": 3 },
        { "name": "Clothes Clothes Clothes", "description": "Discount clothing", "ranking": 4 },
        { "name": "Java Shopping", "description": "Indonesian goods", "ranking": 5 }
    ]
)

In [ ]:
db.articles.drop()

### 8.6. Single(단일) 필드 인덱스 생성 with pymongo

In [ ]:
# text 인덱스 생성 
db.articles.create_index('ranking')

'ranking_1'

In [ ]:
# text 인덱스 확인
db.articles.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_index_db.articles', 'v': 2},
 'ranking_1': {'key': [('ranking', 1)],
  'ns': 'test_index_db.articles',
  'v': 2}}

- "key" which is a list of (key, direction) pairs specifying the index (as passed to create_index()).
- _id 는 기본 인덱스로 설정, 1 은 pymongo.ASCENDING
- subject 는 create_index() 로 만들어진 인덱스
  - 키의 1, -1, 'text' 알아보기
    - pymongo.ASCENDING = 1
    - pymongo.DESCENDING = -1
    - pymongo.TEXT = 'text'

### 8.7. 특정 인덱스 삭제 with pymongo

In [ ]:
# index_information() 에 넣어진 'key'의 값을 통째로 정확하게 적어야 삭제 가능
db.articles.drop_index([('ranking', 1)])

In [ ]:
# text 인덱스 다시 생성 
db.articles.create_index([('ranking', pymongo.DESCENDING)])
db.articles.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_index_db.articles', 'v': 2},
 'ranking_-1': {'key': [('ranking', -1)],
  'ns': 'test_index_db.articles',
  'v': 2}}

### 8.8. 전체 인덱스 삭제 with pymongo

In [ ]:
# index_information() 에 넣어진 'key'의 값을 통째로 정확하게 적어야 삭제 가능
db.articles.drop_indexes()

In [ ]:
db.articles.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_index_db.articles', 'v': 2}}

### 8.9. Text 인덱스 생성 with pymongo

In [ ]:
db.articles.create_index([('name', pymongo.TEXT)])

'name_text'

In [ ]:
db.articles.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_index_db.articles', 'v': 2},
 'name_text': {'default_language': 'english',
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'language_override': 'language',
  'ns': 'test_index_db.articles',
  'textIndexVersion': 3,
  'v': 2,
  'weights': SON([('name', 1)])}}

In [ ]:
db.articles.drop_indexes()

### 8.10. Compound (복합)  필드 인덱스 생성 with pymongo

In [ ]:
db.articles.create_index([('name', pymongo.TEXT), ('ranking', pymongo.ASCENDING)])

'name_text_ranking_1'

In [ ]:
db.articles.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'test_index_db.articles', 'v': 2},
 'name_text_ranking_1': {'default_language': 'english',
  'key': [('_fts', 'text'), ('_ftsx', 1), ('ranking', 1)],
  'language_override': 'language',
  'ns': 'test_index_db.articles',
  'textIndexVersion': 3,
  'v': 2,
  'weights': SON([('name', 1)])}}

- name_text_ranking_1 : 복합 인덱스는 복합 컬럼으로 검색할 경우에만 의미가 있고, 검색 형태에 따라 성능 개선의 효과가 있을 수도/없을 수도 있음
  - createIndex({a : 1, b : -1})로 생성한 경우
    - db.x.find({a : 1, b : -1}), db.x.find({a : -1, b : 1}) 의 쿼리만 효과를 봄
    - db.x.find({a : 1, b : 1}), db.x.find({a : -1, b : -1}) 의 쿼리는 효과 없음

In [ ]:
db.articles.drop_indexes()

### 8.11. Text 인덱스와 검색 with pymongo

In [ ]:
db.articles.create_index([('name', pymongo.TEXT)])

'name_text'

In [ ]:
result = db.articles.find()
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b0'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}
{'_id': ObjectId('5a0c189581f640052dfee6b1'), 'name': 'Burger Buns', 'description': 'Java hamburgers', 'ranking': 2}
{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('5a0c189581f640052dfee6b3'), 'name': 'Clothes Clothes Clothes', 'description': 'Discount clothing', 'ranking': 4}
{'_id': ObjectId('5a0c189581f640052dfee6b4'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


- 인덱스 데이터에 대해, 텍스트 검색을 하기 위해, $text operator를 사용할 수 있음
- $text operator는 인덱스 텍스트 데이터를 스페이스와 구두점(. ,등)으로 텍스트를 구분하고, 각 텍스트를 OR 로 검색
- 기본적으로 대소문자 구별하지 않음
- name 컬럼이 pymongo.TEXT 로 인덱싱되어, $text operator 로 검색시 name 컬럼만 검색

In [ ]:
# $text operator를 사용한 인덱스 텍스트 데이터 검색 ($text operator 는 $search operator 와 함께 사용됨)
result = db.articles.find({'$text' : {'$search' : 'coffee'}})
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [ ]:
# 띄어쓰기가 있는 경우
result = db.articles.find({'$text' : {'$search' : 'java coffee shop'}})
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b0'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}
{'_id': ObjectId('5a0c189581f640052dfee6b4'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}
{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [ ]:
# coffee shop 으로 정확한 검색
result = db.articles.find({'$text': {'$search':"\"coffee shop\"" } } )
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [ ]:
# 대소문자 구별 (실제 name의 컬럼값은 Coffee 이므로, 검색이 안됨)
result = db.articles.find({'$text' : {'$search' : 'coffee', '$caseSensitive' : True}})
for record in result:
    print(record)

In [ ]:
# 정규표현식 ($text operator 는 $search operator 와 함께 사용됨)
result = db.articles.find({'name' : {'$regex' : 'Cof.+'}})
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [ ]:
# 정규표현식, 실제 컬럼명과 함께 사용 가능 ($text operator 는 $search operator 와 함께 사용됨)
result = db.articles.find({'name' : {'$regex' : 'Cof.+'}})
for record in result:
    print(record)

{'_id': ObjectId('5a0c189581f640052dfee6b2'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
